In [1]:
import pandas as pd
import numpy as np

In [2]:
path = ""

In [3]:
baseline=pd.read_csv(path+'c/maxcut/baseline.csv') #[['name','limit']]
baseline = baseline[baseline['limit'] > 0.25]
#baseline = baseline[baseline['nodes'] > 20]
#baseline

In [4]:
len(baseline)

2125

In [5]:
results = pd.read_csv(path+'/data/results.csv')
len(results)

1895665

In [6]:
# check if there are more than 5 unique seeds per heuristic and instance
run_counts = results.groupby(["heuristic", "graphname"])["seed"].nunique().reset_index()
run_counts[run_counts['seed'] != 5]

,heuristic,graphname,seed


In [7]:
# check if there are seeds other than [144, 244, 344, 444, 544]

allowed_seeds = [144, 244, 344, 444, 544]

unexpected_seeds = results.loc[~results['seed'].isin(allowed_seeds), 'seed'].unique()

print(unexpected_seeds)

[]


In [8]:
# check and print if there are duplicate seeds per heuristic per instance

duplicate_counts = (
    results.groupby(["heuristic", "graphname", "seed"])
    .size()
    .reset_index(name="count")
)

duplicates = duplicate_counts[duplicate_counts["count"] > 1]

print(duplicates)

           heuristic          graphname  seed  count
16375   ALKHAMIS1998  imgseg_100075.zip   144    423
16376   ALKHAMIS1998  imgseg_100075.zip   244    349
16377   ALKHAMIS1998  imgseg_100075.zip   344    412
16378   ALKHAMIS1998  imgseg_100075.zip   444    365
16379   ALKHAMIS1998  imgseg_100075.zip   544    438
...              ...                ...   ...    ...
628650  PARDALOS2008   imgseg_97017.zip   144      4
628651  PARDALOS2008   imgseg_97017.zip   244      3
628652  PARDALOS2008   imgseg_97017.zip   344      2
628653  PARDALOS2008   imgseg_97017.zip   444      3
628654  PARDALOS2008   imgseg_97017.zip   544      5

[18972 rows x 4 columns]


In [9]:
results.groupby("heuristic")["graphname"].nunique()

heuristic
ALKHAMIS1998           3396
BEASLEY1998SA          3396
BEASLEY1998TS          3396
BURER2002              3396
DESOUSA2013             100
DUARTE2005             3396
FESTA2002G             3396
FESTA2002GPR           3396
FESTA2002GVNS          3396
FESTA2002GVNSPR        3396
FESTA2002VNS           3396
FESTA2002VNSPR         3396
GLOVER1998a            3396
GLOVER2010             3396
HASAN2000GA            3396
HASAN2000TS            3396
KATAYAMA2000           3396
KATAYAMA2001           3396
LAGUNA2009CE           3396
LAGUNA2009HCE          3396
LODI1999               3396
LU2010                 3396
MERZ1999CROSS          3396
MERZ1999GLS            3396
MERZ1999MUTATE         3396
MERZ2002GREEDY         3396
MERZ2002GREEDYKOPT     3396
MERZ2002KOPT           3396
MERZ2002ONEOPT         3396
MERZ2004               3396
PALUBECKIS2004bMST1    3396
PALUBECKIS2004bMST2    3396
PALUBECKIS2004bMST3    3396
PALUBECKIS2004bMST4    3396
PALUBECKIS2004bMST5    3396
PALUBECKIS

In [10]:
# are these all image segmentation instances?

not_imgseg = duplicates[~duplicates["graphname"].str.startswith("imgseg")]

print(not_imgseg) #yes

Empty DataFrame
Columns: [heuristic, graphname, seed, count]
Index: []


In [11]:
imgseg = results[results['graphname'].str.startswith("imgseg")]

print("total of image segmentation instances:", len(imgseg['graphname'].unique()))
print("Total number of runs for image sementation", len(imgseg), "those with far off runtimes from limit:", len(imgseg[imgseg['runtime'] / imgseg['limit'] < 0.6 ]))

#I suggest we restrict image segmentation data to those were the runtime is 0.99 of the limit


#len(imgseg[imgseg['runtime'] / imgseg['limit'] > 0.99 ]['heuristic'].unique()) #38 -> good
#len(imgseg[imgseg['runtime'] / imgseg['limit'] > 0.99 ]['graphname'].unique()) #100 -> good

# we make sure that at least one run is present for each heuristics
len(imgseg[imgseg['runtime'] / imgseg['limit'] > 0.99 ].groupby("heuristic")["graphname"].nunique()) #38 -> good
imgseg[(imgseg['runtime'] / imgseg['limit']) > 0.99 ].groupby("heuristic")["graphname"].nunique()

total of image segmentation instances: 100
Total number of runs for image sementation 1285905 those with far off runtimes from limit: 1015409


heuristic
ALKHAMIS1998             3
BEASLEY1998SA            4
BEASLEY1998TS            5
BURER2002                6
DESOUSA2013             16
DUARTE2005              15
FESTA2002G               2
FESTA2002GPR             8
FESTA2002GVNS            2
FESTA2002GVNSPR          6
FESTA2002VNS             3
FESTA2002VNSPR           4
GLOVER1998a              1
GLOVER2010              25
HASAN2000GA             55
HASAN2000TS              3
KATAYAMA2000             6
KATAYAMA2001             2
LAGUNA2009CE             2
LAGUNA2009HCE           14
LODI1999                 7
LU2010                  13
MERZ1999CROSS           96
MERZ1999GLS             36
MERZ1999MUTATE         100
MERZ2002GREEDY           4
MERZ2002GREEDYKOPT       2
MERZ2002KOPT             6
MERZ2002ONEOPT           4
MERZ2004                30
PALUBECKIS2004bMST1      5
PALUBECKIS2004bMST2     19
PALUBECKIS2004bMST3      3
PALUBECKIS2004bMST4      8
PALUBECKIS2004bMST5      7
PALUBECKIS2004bSTS       3
PALUBECKIS2006    

In [12]:
# Is 'DESOUSA2013' run on other instances than image segmentation? 
results[results['heuristic'] == 'DESOUSA2013']['graphname'].unique() #No

array(['imgseg_271031.zip', 'imgseg_80099.zip', 'imgseg_106025.zip',
       'imgseg_374067.zip', 'imgseg_22090.zip', 'imgseg_105019.zip',
       'imgseg_285036.zip', 'imgseg_271008.zip', 'imgseg_28096.zip',
       'imgseg_183055.zip', 'imgseg_365025.zip', 'imgseg_55075.zip',
       'imgseg_43083.zip', 'imgseg_147021.zip', 'imgseg_20008.zip',
       'imgseg_100075.zip', 'imgseg_353013.zip', 'imgseg_370036.zip',
       'imgseg_163014.zip', 'imgseg_253036.zip', 'imgseg_310007.zip',
       'imgseg_249061.zip', 'imgseg_374020.zip', 'imgseg_35058.zip',
       'imgseg_42044.zip', 'imgseg_140075.zip', 'imgseg_166081.zip',
       'imgseg_22013.zip', 'imgseg_232038.zip', 'imgseg_108073.zip',
       'imgseg_65132.zip', 'imgseg_292066.zip', 'imgseg_15088.zip',
       'imgseg_242078.zip', 'imgseg_216041.zip', 'imgseg_65074.zip',
       'imgseg_293029.zip', 'imgseg_35010.zip', 'imgseg_12003.zip',
       'imgseg_45077.zip', 'imgseg_2092.zip', 'imgseg_176039.zip',
       'imgseg_187003.zip', 'imgseg_6

In [13]:
# Thus, I suggest we drop this solver
results = results[results['heuristic'] != 'DESOUSA2013']

In [14]:
unique_heuristics = results['heuristic'].unique()

results['graphname'] = results['graphname'].str.replace(r'\.zip$', '', regex=True)
results = results.rename(columns={'graphname':'name'})

#Keeps only rows that have the maximum objective per name (keeps multiple runs per heuristics)
results = results[results['objective'] == results.groupby('name')['objective'].transform('max')]

In [15]:
#MQLib did not stay consistant with recording the actual runtime 
#so we don't look at it

In [16]:
len(unique_heuristics)

37

In [17]:
#List all heuristics that found the best objectiv value for each instance
aggregated_results = results.groupby('name').agg(
    objective=('objective', 'max'),
    heuristic=('heuristic', lambda x: ', '.join(sorted(set(x))))  
).reset_index()
aggregated_results

,name,objective,heuristic
0,G55,10249.0,BURER2002
1,G56,3982.0,BURER2002
2,G57,3424.0,BURER2002
3,G58,19139.0,ALKHAMIS1998
4,G59,5959.0,MERZ1999GLS
...,...,...,...
3391,p6000_2,14333855.0,"GLOVER2010, LU2010, PALUBECKIS2004bMST2, PALUB..."
3392,p6000_3,16132915.0,"GLOVER2010, LU2010, MERZ2004, PALUBECKIS2004bM..."
3393,p7000_1,14478676.0,"LU2010, PALUBECKIS2004bMST2, PALUBECKIS2006"
3394,p7000_2,18249948.0,"GLOVER2010, LU2010, PALUBECKIS2006"


In [18]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v and v != 0 else '' for v in is_max]

In [19]:
aggregated_results=pd.merge(aggregated_results,baseline[['name', 'nodes','edges','density']], on='name', how='right')

In [20]:
len(aggregated_results)

2125

In [21]:
aggregated_results = aggregated_results.rename(columns={'nodes':'n', 'edges':'m' })

In [22]:
all_heuristics = aggregated_results['heuristic'].str.split(', ').explode()

heuristic_counts = all_heuristics.value_counts()

heuristic_counts

heuristic
PALUBECKIS2004bMST2    702
BURER2002              592
MERZ1999GLS            569
PALUBECKIS2004bMST3    549
MERZ2004               496
LU2010                 485
PALUBECKIS2006         463
FESTA2002GVNSPR        460
PALUBECKIS2004bMST5    445
GLOVER2010             443
BEASLEY1998TS          442
PALUBECKIS2004bMST1    434
FESTA2002GPR           413
FESTA2002GVNS          407
PALUBECKIS2004bMST4    384
PALUBECKIS2004bSTS     353
FESTA2002G             339
MERZ2002GREEDYKOPT     325
MERZ2002KOPT           303
PARDALOS2008           294
ALKHAMIS1998           284
KATAYAMA2000           283
FESTA2002VNSPR         281
FESTA2002VNS           281
DUARTE2005             211
HASAN2000TS            210
HASAN2000GA            195
KATAYAMA2001           183
LAGUNA2009HCE          175
MERZ2002ONEOPT         148
BEASLEY1998SA          146
GLOVER1998a            131
MERZ2002GREEDY         121
LODI1999                80
LAGUNA2009CE            33
MERZ1999CROSS           33
MERZ1999MUTATE    

In [23]:
data = aggregated_results
#data = data.dropna()

In [24]:
#data=aggregated_results
mask= (data['n']<4096) & (data['n']>=2048)
maskl= (data['n']>=4096) & (data['n']<8192)
data_xsmall = data[(data['n']<1024) & (data['n']>=20)]
data_small=data[(data['n']<2048) & (data['n']>=1024)]
data_medium=data[mask]
data_large=data[maskl]
data_xlarge=data[data['n']>=8192]

print('xsmall', len(data_xsmall))
print('small', len(data_small))
print('medium', len(data_medium))
print('large', len(data_large))
print('xlarge', len(data_xlarge))
print('TOTAL', len(data_small) + len(data_medium) + len(data_large) + len(data_xlarge))

xsmall 1074
small 232
medium 407
large 331
xlarge 81
TOTAL 1051


In [25]:
all_xsmall = data_xsmall['heuristic'].str.split(', ').explode()
all_small = data_small['heuristic'].str.split(', ').explode()
all_medium = data_medium['heuristic'].str.split(', ').explode()
all_large = data_large['heuristic'].str.split(', ').explode()
all_xlarge = data_xlarge['heuristic'].str.split(', ').explode()

heuristic_counts_xsmall = all_xsmall.value_counts()
heuristic_counts_small = all_small.value_counts()
heuristic_counts_medium = all_medium.value_counts()
heuristic_counts_large = all_large.value_counts()
heuristic_counts_xlarge = all_xlarge.value_counts()

In [26]:
full_table = pd.merge(pd.merge(heuristic_counts_xsmall, heuristic_counts_small, on='heuristic',how='outer'), heuristic_counts_medium, on='heuristic',how='outer')
full_table = full_table.rename(columns={'count_x':'count_xsmall'})
full_table = full_table.rename(columns={'count_y':'count_small'})
full_table = full_table.rename(columns={'count':'count_medium'})
full_table = pd.merge(full_table, heuristic_counts_large, on='heuristic',how='outer')
full_table = full_table.rename(columns={'count':'count_large'})
full_table = pd.merge(full_table, heuristic_counts_xlarge, on='heuristic',how='outer')
full_table = full_table.rename(columns={'count':'count_xlarge'})

full_table.iloc[:, -1] = full_table.iloc[:, -1].fillna(0)
full_table[full_table.columns[-1]] = full_table[full_table.columns[-1]].astype(int)
full_table.style.apply(highlight_max)

,count_xsmall,count_small,count_medium,count_large,count_xlarge
heuristic,,,,,
PALUBECKIS2004bMST2,402,40,132,125,3
PALUBECKIS2004bMST3,354,44,94,47,10
BEASLEY1998TS,342,23,51,26,0
LU2010,334,31,71,49,0
MERZ1999GLS,330,47,73,82,37
PALUBECKIS2006,326,28,64,44,1
PALUBECKIS2004bMST1,321,23,55,35,0
PALUBECKIS2004bMST5,319,33,64,29,0
GLOVER2010,301,27,67,45,3


In [27]:
value_sparse = 0.1
value_dense = 0.5

data_xsmall_sparse=data_xsmall[data_xsmall['density']<value_sparse]
data_xsmall_med=data_xsmall[(data_xsmall['density']<value_dense) & (data_xsmall['density']>=value_sparse)]
data_xsmall_dense=data_xsmall[data_xsmall['density']>=value_dense]

print('small_sparse', len(data_xsmall_sparse))
print('small_med', len(data_xsmall_med))
print('small_dense', len(data_xsmall_dense))
print()

data_small_sparse=data_small[data_small['density']<value_sparse]
data_small_med=data_small[(data_small['density']<value_dense) & (data_small['density']>=value_sparse)]
data_small_dense=data_small[data_small['density']>=value_dense]

print('small_sparse', len(data_small_sparse))
print('small_med', len(data_small_med))
print('small_dense', len(data_small_dense))
print()

data_med_sparse=data_medium[data_medium['density']<value_sparse]
data_med_med=data_medium[(data_medium['density']<value_dense) & (data_medium['density']>=value_sparse)]
data_med_dense=data_medium[data_medium['density']>=value_dense]

print('med_sparse', len(data_med_sparse))
print('med_med', len(data_med_med))
print('med_dense', len(data_med_dense))
print()

data_large_sparse=data_large[data_large['density']<value_sparse]
data_large_med=data_large[(data_large['density']<value_dense) & (data_large['density']>=value_sparse)]
data_large_dense=data_large[data_large['density']>=value_dense]

print('large_sparse', len(data_large_sparse))
print('large_med', len(data_large_med))
print('large_dense', len(data_large_dense))
print()


data_xlarge_sparse=data_xlarge[data_xlarge['density']<value_sparse]
data_xlarge_dense=data_xlarge[data_xlarge['density']>=value_dense]
data_xlarge_med=data_xlarge[(data_xlarge['density']<value_dense) & (data_xlarge['density']>=value_sparse)]

print('xlarge_sparse', len(data_xlarge_sparse))
print('xlarge_med', len(data_xlarge_med))
print('xlarge_dense', len(data_xlarge_dense))
print()

small_sparse 525
small_med 367
small_dense 182

small_sparse 179
small_med 41
small_dense 12

med_sparse 279
med_med 78
med_dense 50

large_sparse 171
large_med 119
large_dense 41

xlarge_sparse 81
xlarge_med 0
xlarge_dense 0



In [28]:
all_xsmall_sparse = data_xsmall_sparse['heuristic'].str.split(', ').explode()
all_xsmall_med = data_xsmall_med['heuristic'].str.split(', ').explode()
all_xsmall_dense = data_xsmall_dense['heuristic'].str.split(', ').explode()

all_small_sparse = data_small_sparse['heuristic'].str.split(', ').explode()
all_small_med = data_small_med['heuristic'].str.split(', ').explode()
all_small_dense = data_small_dense['heuristic'].str.split(', ').explode()

all_med_sparse = data_med_sparse['heuristic'].str.split(', ').explode()
all_med_med = data_med_med['heuristic'].str.split(', ').explode()
all_med_dense = data_med_dense['heuristic'].str.split(', ').explode()

all_large_sparse = data_large_sparse['heuristic'].str.split(', ').explode()
all_large_med = data_large_med['heuristic'].str.split(', ').explode()
all_large_dense = data_large_dense['heuristic'].str.split(', ').explode()

all_xlarge_sparse = data_xlarge_sparse['heuristic'].str.split(', ').explode()
all_xlarge_dense = data_xlarge_dense['heuristic'].str.split(', ').explode()
all_xlarge_med = data_xlarge_med['heuristic'].str.split(', ').explode()

xs_counts_sparse = all_xsmall_sparse.value_counts()
xs_counts_med = all_xsmall_med.value_counts()
xs_counts_dense = all_xsmall_dense.value_counts()

s_counts_sparse = all_small_sparse.value_counts()
s_counts_med = all_small_med.value_counts()
s_counts_dense = all_small_dense.value_counts()

m_counts_sparse = all_med_sparse.value_counts()
m_counts_med = all_med_med.value_counts()
m_counts_dense = all_med_dense.value_counts()

l_counts_sparse = all_large_sparse.value_counts()
l_counts_med = all_large_med.value_counts()
l_counts_dense = all_large_dense.value_counts()

xl_counts_sparse = all_xlarge_sparse.value_counts()
xl_counts_dense = all_xlarge_dense.value_counts()
xl_counts_med = all_xlarge_med.value_counts()


In [29]:
df_xs_counts_sparse = xs_counts_sparse.rename('xsmall_sparse('+str(len(data_xsmall_sparse))+')')
df_xs_counts_med = xs_counts_med.rename('xsmall_med('+str(len(data_xsmall_med))+')')
df_xs_counts_dense = xs_counts_dense.rename('xsmall_dense('+str(len(data_xsmall_dense))+')')

df_s_counts_sparse = s_counts_sparse.rename('small_sparse('+str(len(data_small_sparse))+')')
df_s_counts_med = s_counts_med.rename('small_med('+str(len(data_small_med))+')')
df_s_counts_dense = s_counts_dense.rename('small_dense('+str(len(data_small_dense))+')')

df_m_counts_sparse = m_counts_sparse.rename('medium_sparse('+str(len(data_med_sparse))+')')
df_m_counts_med = m_counts_med.rename('medium_med('+str(len(data_med_med))+')')
df_m_counts_dense = m_counts_dense.rename('medium_dense('+str(len(data_med_dense))+')')

df_l_counts_sparse = l_counts_sparse.rename('large_sparse('+str(len(data_large_sparse))+')')
df_l_counts_med = l_counts_med.rename('large_med('+str(len(data_large_med))+')')
df_l_counts_dense = l_counts_dense.rename('large_dense('+str(len(data_large_dense))+')')

df_xl_counts_sparse = xl_counts_sparse.rename('sparse_xlarge('+str(len(data_xlarge_sparse))+')')
df_xl_counts_dense = xl_counts_dense.rename('dense_xlarge('+str(len(data_xlarge_dense))+')')
df_xl_counts_med = xl_counts_med.rename('med_xlarge('+str(len(data_xlarge_med))+')')

# Concatenate all dataframes into a single one, aligning on the index (heuristic)
merged_df = pd.concat([ 
    df_xs_counts_sparse, df_xs_counts_med, df_xs_counts_dense,
    df_s_counts_sparse, df_s_counts_med, df_s_counts_dense,
    df_m_counts_sparse, df_m_counts_med, df_m_counts_dense,
    df_l_counts_sparse, df_l_counts_med, df_l_counts_dense,
    df_xl_counts_sparse, df_xl_counts_med, df_xl_counts_dense
], axis=1)

# Fill NaN values with 0 if needed (optional)
merged_df = merged_df.fillna(0)
merged_df.style.apply(highlight_max)

merged_df_numeric = merged_df.select_dtypes(include=['number']).astype(int)

styled_df = merged_df_numeric.style.apply(lambda x: ['background-color: yellow' if v == x.max() else '' for v in x], axis=0)
styled_df

,xsmall_sparse(525),xsmall_med(367),xsmall_dense(182),small_sparse(179),small_med(41),small_dense(12),medium_sparse(279),medium_med(78),medium_dense(50),large_sparse(171),large_med(119),large_dense(41),sparse_xlarge(81),med_xlarge(0),dense_xlarge(0)
heuristic,,,,,,,,,,,,,,,
BURER2002,205,70,23,85,0,2,111,10,4,56,12,4,10,0,0
MERZ1999GLS,138,149,43,36,8,3,41,25,7,55,23,4,37,0,0
FESTA2002GVNSPR,137,96,34,52,1,3,61,11,5,28,14,4,14,0,0
FESTA2002GPR,124,58,30,65,1,3,63,11,4,26,13,4,11,0,0
FESTA2002GVNS,117,91,28,43,1,3,54,11,4,25,14,4,12,0,0
FESTA2002G,112,38,11,49,0,3,59,8,4,25,11,4,15,0,0
PALUBECKIS2004bMST3,107,150,97,37,4,3,67,14,13,23,19,5,10,0,0
PALUBECKIS2004bMST2,100,183,119,21,13,6,66,39,27,31,66,28,3,0,0
PALUBECKIS2004bMST5,70,154,95,21,8,4,34,14,16,6,15,8,0,0,0


In [30]:
from itertools import product
sizes = ['xs','s','m','l','xl']
densities = ['sparse','med','dense']

for size, density in product(sizes, densities):
    var_name = f"df_{size}_counts_{density}"

    print(f"\nContents of {var_name}:")
    print(list(globals()[var_name].index[:1]))


Contents of df_xs_counts_sparse:
['BURER2002']

Contents of df_xs_counts_med:
['PALUBECKIS2004bMST2']

Contents of df_xs_counts_dense:
['PALUBECKIS2004bMST2']

Contents of df_s_counts_sparse:
['BURER2002']

Contents of df_s_counts_med:
['PALUBECKIS2004bMST2']

Contents of df_s_counts_dense:
['PALUBECKIS2004bMST2']

Contents of df_m_counts_sparse:
['BURER2002']

Contents of df_m_counts_med:
['PALUBECKIS2004bMST2']

Contents of df_m_counts_dense:
['PALUBECKIS2004bMST2']

Contents of df_l_counts_sparse:
['BURER2002']

Contents of df_l_counts_med:
['PALUBECKIS2004bMST2']

Contents of df_l_counts_dense:
['PALUBECKIS2004bMST2']

Contents of df_xl_counts_sparse:
['MERZ1999GLS']

Contents of df_xl_counts_med:
[]

Contents of df_xl_counts_dense:
[]
